In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
import numpy as np
import matplotlib.pyplot as plt

# 设置随机种子
torch.manual_seed(42)

# 定义是否使用CUDA
use_cuda = True
device = torch.device("cuda" if use_cuda and torch.cuda.is_available() else "cpu")

# 定义epsilons
epsilons = [0, .05, .1, .15, .2, .25, .3]

# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# 加载CIFAR-10数据集
trainset = datasets.CIFAR10(root='../data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='../data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=8)
test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=True, num_workers=8)

# 定义ResNet18模型
model = models.resnet18(pretrained=False, num_classes=10)
model = model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# 定义训练函数
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# 定义FGSM攻击函数
def fgsm_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon*sign_data_grad
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

# 定义测试函数
def test(model, device, test_loader, epsilon):
    correct = 0
    adv_examples = []

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        data.requires_grad = True

        output = model(data)
        init_pred = output.max(1, keepdim=True)[1]

        if init_pred.item() != target.item():
            continue

        loss = criterion(output, target)
        model.zero_grad()
        loss.backward()
        data_grad = data.grad.data

        perturbed_data = fgsm_attack(data, epsilon, data_grad)
        output = model(perturbed_data)

        final_pred = output.max(1, keepdim=True)[1]
        if final_pred.item() == target.item():
            correct += 1
            if epsilon == 0 and len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append((init_pred.item(), final_pred.item(), adv_ex))
        else:
            if len(adv_examples) < 5:
                adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
                adv_examples.append((init_pred.item(), final_pred.item(), adv_ex))

    final_acc = correct / float(len(test_loader))
    print(f"Epsilon: {epsilon}\tTest Accuracy = {correct} / {len(test_loader)} = {final_acc}")

    return final_acc, adv_examples

# 训练和测试
num_epochs = 10
for epoch in range(num_epochs):
    train(model, device, train_loader, optimizer, epoch)

accuracies = []
examples = []

# FGSM测试每个epsilon
for eps in epsilons:
    acc, ex = test(model, device, test_loader, eps)
    accuracies.append(acc)
    examples.append(ex)

# 绘制结果
plt.figure(figsize=(5,5))
plt.plot(epsilons, accuracies, "*-")
plt.yticks(np.arange(0, 1.1, step=0.1))
plt.xticks(np.arange(0, .35, step=0.05))
plt.title("Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.show()


Files already downloaded and verified
Files already downloaded and verified


/home/fu/miniconda3/envs/dbps/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/fu/miniconda3/envs/dbps/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Train Epoch: 0 [0/50000 (0%)]	Loss: 2.583540
Train Epoch: 0 [12800/50000 (26%)]	Loss: 2.222204
Train Epoch: 0 [25600/50000 (51%)]	Loss: 1.741618
Train Epoch: 0 [38400/50000 (77%)]	Loss: 1.722695
Train Epoch: 1 [0/50000 (0%)]	Loss: 1.544039
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.480816
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.273900
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.316620
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.159422
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.206190
Train Epoch: 2 [25600/50000 (51%)]	Loss: 1.130682
Train Epoch: 2 [38400/50000 (77%)]	Loss: 1.165845
Train Epoch: 3 [0/50000 (0%)]	Loss: 1.005356
Train Epoch: 3 [12800/50000 (26%)]	Loss: 0.955615
Train Epoch: 3 [25600/50000 (51%)]	Loss: 0.959892
Train Epoch: 3 [38400/50000 (77%)]	Loss: 0.974417
Train Epoch: 4 [0/50000 (0%)]	Loss: 0.893669
Train Epoch: 4 [12800/50000 (26%)]	Loss: 0.999458
Train Epoch: 4 [25600/50000 (51%)]	Loss: 0.838141
Train Epoch: 4 [38400/50000 (77%)]	Loss: 0.845951
Train Epoch: 5 [0/50000 (

RuntimeError: a Tensor with 100 elements cannot be converted to Scalar